In [3]:
import os
from pathlib import Path
from gurobipy import Model, GRB
from data_loader import load_prices, load_storage, load_efficiency, load_plant_capacity, load_demand

# Data Loading from data folder in the parent directory
current_dir = Path().resolve()
data_folder = os.path.join(current_dir.parent, "data")
fuel_prices_file = os.path.join(data_folder, "fuel_prices.csv")           # Price in EUR/tonne (coal,oil) or EUR/m3 (gas)
storage_file = os.path.join(data_folder, "storage.csv")         # Max storage in tonnes/m3
efficiency_file = os.path.join(data_folder, "efficiency.csv")   # efficiency of MWh th to MWh el
plant_file = os.path.join(data_folder, "plant_capacity.csv")    # Plant capacity in MWh/month
demand_file = os.path.join(data_folder, "electricity_demand.csv")           # Monthly demand in MWh

start_date = "2020-01-01"
end_date = "2020-12-31"

fuels = ["coal", "oil", "gas"]
zone = 'DK_2'

try:
    fuel_prices_df = load_prices(fuel_prices_file, start_date, end_date)
    storage = load_storage(storage_file)
    efficiency = load_efficiency(efficiency_file)
    plant_capacity = load_plant_capacity(plant_file)
    demand_df = load_demand(demand_file, start_date, end_date, zone=zone ,supply_factor=0.0005)
except Exception as e:
    raise RuntimeError(f"Error loading data: {e}")


# Model Initialization

m = Model("Fuel_Procurement_OneMonth")

# Decision Variables
x = {fuel: m.addVar(lb=0, ub=storage[fuel], name=f"buy_{fuel}") for fuel in fuel_prices_df.columns if fuel in fuels}
y = {fuel: m.addVar(lb=0, ub=plant_capacity[fuel], name=f"gen_{fuel}") for fuel in fuel_prices_df.columns if fuel in fuels}

m.update()

# Constraints
m.addConstr(sum(y[fuel] for fuel in y) >= demand_df.iloc[0][zone], "Demand_Constraint")
for fuel in y:
    m.addConstr(y[fuel] <= efficiency[fuel] * x[fuel], f"efficiency_Constraint_{fuel}")
    # Print the added constraint for verification
    print(f"Added constraint: gen_{fuel} <= {efficiency[fuel]} * buy_{fuel}")
m.update()

# Objective
m.setObjective(sum(fuel_prices_df.iloc[0][fuel] * x[fuel] for fuel in x),GRB.MINIMIZE) 
m.update()

# Optimize
m.optimize()

# print results
if m.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    for fuel in x:
        print(f"Purchased {x[fuel].X} units of {fuel}")
    for fuel in y:
        print(f"Generated {y[fuel].X} units of electricity from {fuel}")
    for fuel in x:
        cost = fuel_prices_df.iloc[0][fuel] * x[fuel].X
        print(f"Cost for {fuel}: EUR {cost}")
    total_cost = m.objVal
    print(f"Total Cost: EUR {total_cost}")
else:
    print("No optimal solution found.")


Added constraint: gen_coal <= 0.38 * buy_coal
Added constraint: gen_oil <= 0.33 * buy_oil
Added constraint: gen_gas <= 0.55 * buy_gas
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 6 columns and 9 nonzeros
Model fingerprint: 0x7ad8e0d3
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [6e+00, 5e+01]
  Bounds range     [4e+02, 5e+03]
  RHS range        [6e+02, 6e+02]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.637925e+01   0.000000e+00      0s
       1    9.4485451e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal ob